# Functional API

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns

# crps loss function 
def crps(y_true, S):
    """
    Computes continuous ranked probability score:

    Parameters
    ----------
    y_true : tf tensor of shape (BATCH_SIZE, 1)
        True values.
    S : tf tensor of shape (BATCH_SIZE, N_SAMPLES)
        Predictive samples.

    Returns
    -------
    tf tensor of shape (BATCH_SIZE,)
        Scores.

    """
    beta=1
    n_samples = S.shape[-1]

    def expected_dist(diff, beta):
        return K.sum(K.pow(K.sqrt(K.square(diff)+K.epsilon()), beta),axis=-1) #axis = -1: last dimension <=> N_SAMPLES
    es_1 = expected_dist(y_true - S, beta)
    es_2 = 0
    for i in range(n_samples):
        es_2 = es_2 + expected_dist(K.expand_dims(S[:,i]) - S, beta)
    return es_1/n_samples - es_2/(2*n_samples**2)


class CRPSLoss(Loss):
    def call(self, y_true, S):
        return crps(y_true, S)


# Funktion für die ReLU-Transformation
def relu_transform(x):
    return tf.nn.relu(x)


## hyperparameters
batchSize = 1 # defines the number of samples to work through before 
# updating the internal model parameters (sample = (1 inputvector, 1 y_true))
epochSize = 10 # defines the number times that the learning algorithm will work through the entire training dataset
# -> line plots that show epochs along the x-axis as time and the error or skill of the model on the y-axis (= learning curve)
learningRate = 0.001


# Define inputs with predefined shape
input_shape = (len(X_train[0]),) # Number of used features   10 * 32
inputs = Input(shape=input_shape)

#print(inputs.shape) # -> (None, 10, 32) no Batch size defined (more flexible)

hidden_layer1 = Dense(4, activation='relu')(inputs) 
# Dense Layer: the 10 neurons in the dense layer get their source of input data 
# from all the other neurons of the previous layer of the network (= fully connected layer)
hidden_layer2 = Dense(4, activation='relu')(hidden_layer1) 

#
output_s3 = Dense(100)(hidden_layer1)
sample_output_s3 = Lambda(relu_transform)(output_s3)

# Construct model
model = Model(inputs=inputs, outputs=sample_output_s3, name = 'feedfwdNN_empirical')

# Compile the model with the desired optimizer, loss function, etc.
model.compile(optimizer=Adam(learning_rate=learningRate), loss=CRPSLoss())

# Print model summary
model.summary()

# Wrapping ECON

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Input, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns

# crps loss function 
def crps(y_true, S):
    """
    Computes continuous ranked probability score:

    Parameters
    ----------
    y_true : tf tensor of shape (BATCH_SIZE, 1)
        True values.
    S : tf tensor of shape (BATCH_SIZE, N_SAMPLES)
        Predictive samples.

    Returns
    -------
    tf tensor of shape (BATCH_SIZE,)
        Scores.

    """
    beta=1
    n_samples = S.shape[-1]

    def expected_dist(diff, beta):
        return K.sum(K.pow(K.sqrt(K.square(diff)+K.epsilon()), beta),axis=-1) #axis = -1: last dimension <=> N_SAMPLES
    es_1 = expected_dist(y_true - S, beta)
    es_2 = 0
    for i in range(n_samples):
        es_2 = es_2 + expected_dist(K.expand_dims(S[:,i]) - S, beta)
    return es_1/n_samples - es_2/(2*n_samples**2)


class CRPSLoss(Loss):
    def call(self, y_true, S):
        return crps(y_true, S)

# Define custom ReLU activation function
class ReLUTransform(Layer):
    def call(self, inputs):
        return tf.nn.relu(inputs)
    
    
# Funktion für die ReLU-Transformation
def relu_transform(x):
    return tf.nn.relu(x)

# Define the Feed Forward Neural Network subclass
class FeedForwardNN(tf.keras.Model):
    def __init__(self, input_shape):
        super(FeedForwardNN, self).__init__()
        
        self.model = self._build_model(input_shape)



    def _build_model(self, input_shape):
        """
        Defines original IGEP model:
        Variance of dim_out of the latent distributions depend on the ensemble spread.
        See Janke&Steinke (2020): "Probabilistic multivariate electricity price forecasting using implicit generative ensemble post-processing"
        https://arxiv.org/abs/2005.13417

        Returns
        -------
        object
            Keras model.

        """

        inputs = Input(shape=input_shape)
        hidden_layer1 = Dense(4, activation='relu')(inputs) 
        hidden_layer2 = Dense(4, activation='relu')(hidden_layer1) 
        output_s3 = Dense(100)(hidden_layer2)
        sample_output_s3 = Lambda(relu_transform)(output_s3)

        return Model(inputs=inputs, outputs=sample_output_s3, name = 'FeedForwardNN')
    
    def fit(self, x, y, batch_size=1, epochs=10, learning_rate = 0.001, verbose=0, 
            callbacks=None, validation_split=0.0, validation_data=None, sample_weight=None, plot_learning_curve=False):
        """
        Fits the model to traning data.

        Parameters
        ----------

        Returns
        -------
        None.

        """    

################################################################################################################
        self.model.compile(loss=CRPSLoss(), optimizer=Adam(learning_rate))
        self.history = self.model.fit(x=x, 
                                      y=y,
                                      batch_size=batch_size, 
                                      epochs=epochs, 
                                      verbose=verbose, 
                                      callbacks=callbacks, 
                                      validation_split=validation_split, 
                                      validation_data=validation_data,
                                      shuffle=True,
                                      sample_weight=sample_weight)
################################################################################################################
        if verbose > 0:
            keras.utils.plot_model(self.model, show_shapes=True)
            self.model.summary()
        
        if plot_learning_curve:
            learning_curve_plot(self.history.history)

# Define hyperparameters
""" batchSize = 1
epochSize = 10
learningRate = 0.001 """

# Define inputs with predefined shape
input_shape = (len(X_train[0]),)

# Create an instance of the FeedForwardNN model
#model = FeedForwardNN(input_shape=input_shape, classes=100)  # Update 'classes' with the appropriate number
model = FeedForwardNN(input_shape)

